In [3]:
import sys

IN_COLAB = 'google.colab' in sys.modules

if not IN_COLAB:

    from git import Repo

    # Initialize the Git repository object
    repo = Repo(".", search_parent_directories=True)

    # Get the root directory of the Git project
    root_dir = repo.git.rev_parse("--show-toplevel")

    from pathlib import Path

    # Set up path for custom importer modules
    importer_module = root_dir + '/dataloader/'
    sys.path.insert(0, importer_module)

    # Insert here your local path to the dataset
    data_path = '/home/leeoos/Desktop/' #input("Path to the dataset: ")

else:

    from google.colab import drive
    drive.mount('/content/drive')

    # On Colab the path to the module ti fixed once you have
    # corretly set up the project with gitsetup.ipynb
    fixed_path = '/content/drive/MyDrive/Github/visiope/dataloader/'
    sys.path.insert(0, fixed_path)

    # Insert here the path to the dataset on your drive
    data_path = input("Path to the dataset: ")

In [ ]:
import numpy as np
import os
from loader import Ai4MarsImporter, Ai4MarsProcessor, Ai4MarsData

print("Making the npdataset")
data_import = Ai4MarsImporter()
X, y = data_import(path=data_path, num_of_images=1000)

processor = Ai4MarsProcessor()
train_set, test_set, val_set = processor(X, y, [0.60, 0.20, 0.20])

# # set up dataset directory
# save_path = input("Path to save the npdataset: ")

# if not os.path.exists(save_path) : 
#     os.makedirs(save_path)

# save_path_X = save_path + 'X'
# np.save(save_path_X, X)

# save_path_y = save_path + 'y'
# np.save(save_path_y, y)

In [5]:
import torch 

torch.save(train_set, '/home/leeoos/Desktop/dataset/train.pt')
torch.save(test_set, '/home/leeoos/Desktop/dataset/test.pt')
torch.save(val_set, '/home/leeoos/Desktop/dataset/val.pt')

# What should this file do?

1. Be able to call dataloader/loader from wherever it is (LOCAL/COLAB)
2. Create an Ai4Mars dataset with given specifics 
3. Save the said dataset/dataloader on LOCAL runtime or on COLAB runtime
4. Save the said dataset/dataloader on google drive

In [ ]:
import os
import sys
import torch
import numpy as np
from torch.utils.data import DataLoader
from loader import Ai4MarsImporter, Ai4MarsProcessor, Ai4MarsData

COLAB = 'google.colab' in sys.modules
LOCAL = not(COLAB)

if LOCAL:

    from git import Repo

    # Initialize the Git repository object
    repo = Repo(".", search_parent_directories=True)

    # Get the root directory of the Git project
    root_dir = repo.git.rev_parse("--show-toplevel")

    from pathlib import Path

    # Set up path for custom importer modules
    importer_module = root_dir + '/dataloader/'
    sys.path.insert(0, importer_module)

    # Insert here your local path to the dataset
    data_path = '/home/leeoos/Desktop/' #str(input("Path to the dataset: "))
    num_of_images = 200 #int(input("How many images do you want to import: "))
    save_path = '/home/leeoos/Desktop/npdataset/' #str(input("Path to save the npdataset [optional]: "))

    # Import data
    from loader import Ai4MarsImporter, Ai4MarsProcessor, Ai4MarsData

    data_import = Ai4MarsImporter()
    X, y = data_import(path=data_path, num_of_images=num_of_images, save_path=save_path)

    pre_process = False

    if pre_process:
        transform = None
        train_set, test_set, val_set = processor(X, y, percentages=[0.54, 0.26, 0.20], transform=transform)

elif COLAB:

    from google.colab import drive
    drive.mount('/content/drive')

    # On Colab the path to the module ti fixed once you have
    # corretly set up the project with gitsetup.ipynb
    fixed_path = '/content/drive/MyDrive/Github/visiope/dataloader/'
    sys.path.insert(0, fixed_path)

    # Insert here the path to the dataset on your drive
    data_path = input("Path to the dataset: ")
    num_of_images = int(input("How many images do you want to import: "))

    # Decide is you want the raw dataset or the numpy version
    npdataset = True

    if npdataset:

        # Establish npdataset
        npdataset_X = 'X_' + str(num_of_images)               
        npdataset_y = 'y' + npdataset_X[1:]
        npdataset_path = '/content/dataset/'

        if not(os.path.exists(npdataset_path + npdataset_X) and
            os.path.exists(npdataset_path + npdataset_y)):

            import gdown

            # get url of np dataset (temporarerly my drive)
            if num_of_images == 200:
                url_X_200 = 'https://drive.google.com/uc?id=1HLAQgjZbGa3lMzdyIvgykCiUJ6P4OaEX'
                url_y_200 = 'https://drive.google.com/uc?id=1Ue74LEe0WlEnFxRcIl19WyvXR1EyYIsS'

            elif num_of_images == 1000:
                url_X_1000 = 'https://drive.google.com/uc?id=1zvXKK1qc2PNbAMyq0XWfqrhAWB5kO3yO'
                url_y_1000 = 'https://drive.google.com/uc?id=1gmnWAGjZJYt-VIpWRK_cEOXQfFfuFAwe'

            # Download np dataset on runtime env
            gdown.download(url_X_1000, data_path, quiet=False)
            gdown.download(url_y_1000, data_path, quiet=False)

    elif not npdataset:

        from loader import Ai4MarsImporter, Ai4MarsProcessor, Ai4MarsData

        data_import = Ai4MarsImporter()
        X, y = data_import(path=data_path, num_of_images=num_of_images)

else:
    raise Exception('Unknown Environment')